In [48]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time
from sqlalchemy import create_engine
import pymysql
from sqlalchemy.orm import sessionmaker

In [2]:
# 遍历获取JD官网网点信息
def get_JD_siteInfo(urls):
    JD_siteList = []
    for url in urls:
        request = requests.get(url)
        soup = BeautifulSoup(request.content, "lxml").get_text()
        jsonlist = json.loads(soup)
        for i in range(len(jsonlist)):
            JD_siteDict = {}
            JD_siteDict["JD_siteBusinessName"] = jsonlist[i]["siteBusinessName"]
            JD_siteDict["JD_siteProvinceId"] = jsonlist[i]["provinceId"]
            JD_siteDict["JD_siteCityId"] = jsonlist[i]["cityId"]
            JD_siteDict["JD_siteCountyId"] = jsonlist[i]["countyId"]
            JD_siteDict["JD_siteName"] = jsonlist[i]["siteName"]
            if 'address' in jsonlist[i]:
                JD_siteDict["JD_siteAddress"] = jsonlist[i]["address"]
            else:
                pass
            if 'latitude' in jsonlist[i]:
                JD_siteDict["JD_siteLat"] = jsonlist[i]['latitude']
            else:
                pass  
            if 'longitude' in jsonlist[i]:
                JD_siteDict["JD_siteLng"] = jsonlist[i]['longitude']
            else:
                pass       
            if "telephone" in jsonlist[i]:
                JD_siteDict["JD_siteTelephone"] = jsonlist[i]["telephone"]
            else:
                pass
            JD_siteList.append(JD_siteDict)
    return JD_siteList

In [3]:
# 解析高德地图API网址信息
def get_json_gaode(address,ak):
    ak = ak_gaode
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [4]:
# 获取高德地图API地址信息
def get_JD_siteInfo_gaode(address):
    print("开始获取高德地址信息")
    start_time = time.time()
    JD_siteList_gaode = []
    for b in address:
        print(b)
        JD_siteDict_gaode = {}
        try:
            temp=get_json_gaode(b,ak_gaode)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('geocodes' in temp):
                JD_siteDict_gaode["JD_siteLocation_gaode"] = temp['geocodes'][0]['location']
                JD_siteDict_gaode["JD_siteFormatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                JD_siteDict_gaode["JD_siteProvince_gaode"] = temp['geocodes'][0]['province']
                JD_siteDict_gaode["JD_siteCountry_gaode"] = temp['geocodes'][0]['country']
                if ('citycode' in temp):
                    JD_siteDict_gaode["JD_siteCitycode_gaode"] = temp['geocodes'][0]['citycode']
                else:
                    pass
                JD_siteDict_gaode["JD_siteCity_gaode"] = temp['geocodes'][0]['city']
                JD_siteDict_gaode["JD_siteDistrict_gaode"] = temp['geocodes'][0]['district']
                JD_siteDict_gaode["JD_siteLevel_gaode"] = temp['geocodes'][0]['level']
                JD_siteDict_gaode["JD_siteTownship_gaode"] = temp['geocodes'][0]['township']
                JD_siteDict_gaode["JD_siteAdcode_gaode"] = temp['geocodes'][0]['adcode']
                JD_siteDict_gaode["JD_siteStreet_gaode"] = temp['geocodes'][0]['street']
                JD_siteDict_gaode["JD_siteNumber_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        JD_siteList_gaode.append(JD_siteDict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return JD_siteList_gaode

In [5]:
# 解析百度地图API网址信息
def get_json_baidu(address,ak):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式
    ak = ak_baidu
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    return temp

In [6]:
# 获取百度地图API地址信息
def get_JD_siteInfo_baidu(address):
    print("开始获取百度地址信息")
    start_time = time.time()
    JD_siteList_baidu = []
    for b in address:
        print(b)
        JD_siteDict_baidu = {}
        try:
            temp = get_json_baidu(b,ak_baidu)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('result' in temp):
                JD_siteDict_baidu["JD_siteLng_baidu"] = temp['result']['location']['lng']
                JD_siteDict_baidu["JD_siteLat_baidu"] = temp['result']['location']['lat']
                JD_siteDict_baidu["JD_sitePrecise_baidu"] = temp['result']['precise']
                JD_siteDict_baidu["JD_siteConfidence_baidu"] = temp['result']['confidence']
                JD_siteDict_baidu["JD_siteComprehension_baidu"] = temp['result']['comprehension']
                JD_siteDict_baidu["JD_siteLevel_baidu"] = temp['result']['level']
            else:
                pass
        JD_siteList_baidu.append(JD_siteDict_baidu)
    #     time.sleep(1)
    end_time = time.time()
    print("百度总用时：",end_time-start_time)
    return JD_siteList_baidu

In [7]:
province = "福建省"
pinyin = "fujian"
inputpath_urls = "G:\\work\\logistica\\stations\\JDStations\\yuanURL\\yuanURL_" + pinyin + ".xlsx"
inputpath_citys = "G:\\work\\logistica\\stations\\JDStations\\cityId\\" + pinyin + "CityID.xlsx"

In [8]:
# 导入数据
df = pd.read_excel(inputpath_urls)
data_city = pd.read_excel(inputpath_citys)

In [9]:
# ak码
ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'



In [10]:

# 导出数据路径
outpath_JD = "G:\\work\\logistica\\stations\\JDStations\\stations\\website\\"
name_ANSI_JD = "JDstations_" + pinyin + "_JD.xlsx"

outpath_gaode = "G:\\work\\logistica\\stations\\JDStations\\stations\\gaode\\"
name_ANSI_gaode = "JDstations_" + pinyin + "_gaode.xlsx"
name_UTF8_gaode = "JDstations_" + pinyin + "_gaode.csv"
outpath_baidu = "G:\\work\\logistica\\stations\\JDStations\\stations\\baidu\\"
name_ANSI_baidu = "JDstations_" + pinyin + "_baidu.xlsx"
name_UTF8_baidu = "JDstations_" + pinyin + "_baidu.csv"


In [11]:
# 建立url池
urls = []
for a in df["url"]:
    urls.append("http://www.jdwl.com/site/querySiteList?" + a)
print("验证完整URL地址：")
print(urls[0:3])



验证完整URL地址：
['http://www.jdwl.com/site/querySiteList?provinceId=16&cityId=1303&countyId=1305', 'http://www.jdwl.com/site/querySiteList?provinceId=16&cityId=1303&countyId=1308', 'http://www.jdwl.com/site/querySiteList?provinceId=16&cityId=1303&countyId=1309']


In [12]:
# 获取京东官网网点信息
df_JD_siteInfo = pd.DataFrame(get_JD_siteInfo(urls))



In [13]:
# 去除空白地址所在行
print("去除空白地址前：")
print(df_JD_siteInfo['JD_siteAddress'].isnull().value_counts())
df_JD_siteInfo = df_JD_siteInfo.dropna(subset=["JD_siteAddress"])
print("去除空白地址后：")
print(df_JD_siteInfo['JD_siteAddress'].isnull().value_counts())
print("*"*50)
print("查看JD网店及对应城市列名：")
print(df_JD_siteInfo.columns)
print("$"*30)
print(data_city.columns)
print("*"*50)



去除空白地址前：
False    288
Name: JD_siteAddress, dtype: int64
去除空白地址后：
False    288
Name: JD_siteAddress, dtype: int64
**************************************************
查看JD网店及对应城市列名：
Index(['JD_siteAddress', 'JD_siteBusinessName', 'JD_siteCityId',
       'JD_siteCountyId', 'JD_siteLat', 'JD_siteLng', 'JD_siteName',
       'JD_siteProvinceId', 'JD_siteTelephone'],
      dtype='object')
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Index(['cityName', 'cityId'], dtype='object')
**************************************************


In [14]:
# data_city['cityId'] = data_city['cityId'].map(lambda x:str(x))
# df_JD_siteInfo["JD_siteCityId"] = df_JD_siteInfo["JD_siteCityId"].map(lambda x:str(x))
df_JD_siteInfo = pd.merge(df_JD_siteInfo,data_city,how="left",left_on="JD_siteCityId",right_on="cityId")
df_JD_siteInfo.rename(columns={'cityName': 'JD_cityName'}, inplace=True)

df_JD_siteInfo["JD_siteAddress_all"] = province + df_JD_siteInfo['JD_cityName'] + df_JD_siteInfo['JD_siteAddress']
print("验证完整收货地址：")
print(df_JD_siteInfo["JD_siteAddress_all"][0:3])
print("&%"*50)

验证完整收货地址：
0      福建省福州市航城街道朝阳北路万业锦江城3号楼28-30店面
1           福建省福州市福建省福州长乐市湖南镇仙宅村新池桥头
2    福建省福州市福建省福州市长乐市江田镇友爱村开发区聚友小区18号
Name: JD_siteAddress_all, dtype: object
&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%


In [15]:
# 获取京东高德网点信息
df_JD_siteList_gaode = pd.DataFrame(get_JD_siteInfo_gaode(df_JD_siteInfo["JD_siteAddress_all"]))

开始获取高德地址信息
福建省福州市航城街道朝阳北路万业锦江城3号楼28-30店面
福建省福州市福建省福州长乐市湖南镇仙宅村新池桥头
福建省福州市福建省福州市长乐市江田镇友爱村开发区聚友小区18号
福建省福州市福建省福州市长乐市首占镇岱边村新西路79号（岱边村公交总站旁）
福建省福州市福建省长乐市江田镇友爱村聚友小区1号店面
福建省福州市福建省福州市平潭县进城路土地后庄(土地后拱门进100米)
福建省福州市福州市平潭县潭城镇上楼村上楼工业区
福建省福州市福建省平潭县风城乡中湖村土地后79号
福建省福州市景江花园二期D型1层2-4店面(敖峰大桥红绿灯处)
福建省福州市福州市连江县坑园镇坑园村宏达路134号
福建省福州市青塘开发区青龙路103号
福建省福州市福建省福州连江县潘渡乡贵安新天地文化街18-6店面			
福建省福州市罗源县时代大厦到底水泥厂旁边
福建省福州市福建省福州市永泰县樟城镇龙峰园390号（江滨路上风顺汽配旁）
福建省福州市福建省福州市闽清县梅城镇梅西新村412号
福建省福州市 福建省福州市台江区宝龙城市广场后面浦西路149号一楼店面
福建省福州市福建省福州市台江区瀛洲街道灜东路83号红星商住楼2#4#楼连体1层01店面
福建省福州市 福建省福州市台江区宝龙城市广场后面浦西路149号一楼店面
福建省福州市福州市台江区浦东路群升白马郡9号楼01店面
福建省福州市福州市台江区茶亭街道国货西路148号亚华大厦1层11-A、11-B
福建省福州市晋安区胜明花园2#楼4#店面
福建省福州市福建省福州市台江区浦西路149号
福建省福州市福建省福州市鼓楼区白龙支路18号五凤山庄二期16#、19#连接体08号店面
福建省福州市福建省福州市鼓楼区杨桥中路272号一楼店面（为民道口烧鸡店）旁
福建省福州市福建省福州市鼓楼区鼓西街道湖头街91号金典小区1#楼16和17店面
福建省福州市福建省福州市鼓楼区洪山镇原厝路62号仁文大儒世家2#楼1层02店面
福建省福州市福州市鼓楼区洪甘镇洪甘路9号洪山防洪堤商品房1#-2#连接体1层
福建省福州市福州市晋安区茶会小区34座店面
福建省福州市福建省福州市晋安区北三环路201号琴亭信和小区1号楼04店面
福建省福州市福建省福州市晋安区福新支路11号
福建省福州市福州市鼓楼区华林路214号省安华林小区
福建省福州市福州市晋安区象园街道晋安南

高德总用时： 13.271759271621704


In [16]:
# 合并并清洗
df_JD_siteInfo_gaode = pd.concat([df_JD_siteList_gaode,df_JD_siteInfo],axis=1)
df_JD_siteInfo_gaode = df_JD_siteInfo_gaode.drop(columns=['JD_siteLng','JD_siteLat'])
split_gaode = pd.DataFrame((x.split(',') for x in df_JD_siteInfo_gaode['JD_siteLocation_gaode']), index=df_JD_siteInfo_gaode.index, columns=['JD_siteLng_gaode','JD_siteLat_gaode'])
df_JD_siteInfo_gaode = pd.merge(df_JD_siteInfo_gaode,split_gaode,left_index=True,right_index=True)

In [17]:
# 获取京东百度网点信息
df_JD_siteList_baidu = pd.DataFrame(get_JD_siteInfo_baidu(df_JD_siteInfo["JD_siteAddress_all"]))
df_JD_siteInfo_baidu = pd.concat([df_JD_siteList_baidu,df_JD_siteInfo],axis=1)
df_JD_siteInfo_baidu = df_JD_siteInfo_baidu.drop(columns=['JD_siteLng','JD_siteLat'])

开始获取百度地址信息
福建省福州市航城街道朝阳北路万业锦江城3号楼28-30店面
福建省福州市福建省福州长乐市湖南镇仙宅村新池桥头
福建省福州市福建省福州市长乐市江田镇友爱村开发区聚友小区18号
福建省福州市福建省福州市长乐市首占镇岱边村新西路79号（岱边村公交总站旁）
福建省福州市福建省长乐市江田镇友爱村聚友小区1号店面
福建省福州市福建省福州市平潭县进城路土地后庄(土地后拱门进100米)
福建省福州市福州市平潭县潭城镇上楼村上楼工业区
福建省福州市福建省平潭县风城乡中湖村土地后79号
福建省福州市景江花园二期D型1层2-4店面(敖峰大桥红绿灯处)
福建省福州市福州市连江县坑园镇坑园村宏达路134号
福建省福州市青塘开发区青龙路103号
福建省福州市福建省福州连江县潘渡乡贵安新天地文化街18-6店面			
福建省福州市罗源县时代大厦到底水泥厂旁边
福建省福州市福建省福州市永泰县樟城镇龙峰园390号（江滨路上风顺汽配旁）
福建省福州市福建省福州市闽清县梅城镇梅西新村412号
福建省福州市 福建省福州市台江区宝龙城市广场后面浦西路149号一楼店面
福建省福州市福建省福州市台江区瀛洲街道灜东路83号红星商住楼2#4#楼连体1层01店面
福建省福州市 福建省福州市台江区宝龙城市广场后面浦西路149号一楼店面
福建省福州市福州市台江区浦东路群升白马郡9号楼01店面
福建省福州市福州市台江区茶亭街道国货西路148号亚华大厦1层11-A、11-B
福建省福州市晋安区胜明花园2#楼4#店面
福建省福州市福建省福州市台江区浦西路149号
福建省福州市福建省福州市鼓楼区白龙支路18号五凤山庄二期16#、19#连接体08号店面
福建省福州市福建省福州市鼓楼区杨桥中路272号一楼店面（为民道口烧鸡店）旁
福建省福州市福建省福州市鼓楼区鼓西街道湖头街91号金典小区1#楼16和17店面
福建省福州市福建省福州市鼓楼区洪山镇原厝路62号仁文大儒世家2#楼1层02店面
福建省福州市福州市鼓楼区洪甘镇洪甘路9号洪山防洪堤商品房1#-2#连接体1层
福建省福州市福州市晋安区茶会小区34座店面
福建省福州市福建省福州市晋安区北三环路201号琴亭信和小区1号楼04店面
福建省福州市福建省福州市晋安区福新支路11号
福建省福州市福州市鼓楼区华林路214号省安华林小区
福建省福州市福州市晋安区象园街道晋安南

福建省宁德市福建省宁德市蕉城区金涵畲族乡金涵村溪坂40号
福建省宁德市福建省宁德市万安西路一号金港名都A区108，109号店
福建省宁德市宁德市东侨区幸福里9号小区1栋101（顺丰快递隔壁）
福建省宁德市福建宁德市东侨开发区学院路1号宁德师范学院学生服务一条街
福建省宁德市福建省宁德市东侨开发区兰田村
百度总用时： 14.126808166503906


In [18]:
# 最后检查
print("JD官网网点信息：" + str(df_JD_siteInfo.shape))
print("$"*30)
print("高德合并之前：" + str(df_JD_siteList_gaode.shape))
print("$"*30)
print("JD网点高德信息：" + str(df_JD_siteInfo_gaode.shape))
print("*"*30)
print("百度合并之前：" + str(df_JD_siteList_baidu.shape))
print("*"*30)
print("JD网点百度信息：" + str(df_JD_siteInfo_gaode.shape))
print("$"*50)

JD官网网点信息：(288, 12)
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
高德合并之前：(288, 11)
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
JD网点高德信息：(288, 23)
******************************
百度合并之前：(288, 6)
******************************
JD网点百度信息：(288, 23)
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


In [19]:
# 输出
df_JD_siteInfo.to_excel(outpath_JD + name_ANSI_JD,index=False)
df_JD_siteInfo_gaode.to_excel(outpath_gaode + name_ANSI_gaode,index=False)
df_JD_siteInfo_baidu.to_excel(outpath_baidu + name_ANSI_baidu,index=False)
df_JD_siteInfo_gaode.to_csv(outpath_gaode + name_UTF8_gaode,index=False)
df_JD_siteInfo_baidu.to_csv(outpath_baidu + name_UTF8_baidu,index=False)